# Transfert Learning avec ResNet50

In [1]:
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils.np_utils import to_categorical

2023-03-04 17:33:38.359114: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


> Charger les images et assigner les labels (ne pas relancer)

In [3]:
from sklearn import preprocessing

In [ ]:
nom_champions_gen = os.listdir('./images_draft')
noms_champions_gen_modif = [nom_champions_gen[x][:-6] for x in range(len(nom_champions_gen))]
# Enlever .DS
noms_champions_gen_modif = [x for x in noms_champions_gen_modif if x != '.DS']
print(noms_champions_gen_modif)
for champion in noms_champions_gen_modif:
    print(champion)
   # Create a folder for each champion and move the unique image of the champion in it
#    os.mkdir('./images_draft/'+champion)

       


In [ ]:
# Move in each folder the file 'the picture
print(noms_champions_gen_modif)
for champion in noms_champions_gen_modif:
    print(champion)
    os.rename('./images_draft/'+champion+'_0.jpg', './images_draft/'+champion+'/'+champion+'_0.jpg')

In [74]:
# For each folder in the folder images_draft, create duplicates in order to have 5 pictures per champion
for champion in noms_champions_gen_modif:
    for i in range(6, 11):
        # We don't use rename because we want to keep the original picture
        os.system('cp ./images_draft/'+champion+'/'+champion+'_0.jpg ./images_draft/'+champion+'/'+champion+'_'+str(i)+'.jpg')
            


Modèle effectif

In [4]:
data_path = "./images_draft"
data_dir_list = os.listdir(data_path)
for x in data_dir_list:
    if x == '.DS_Store':
        data_dir_list.remove(x)
print(data_dir_list)
num_classes = len(data_dir_list) 
print(num_classes)

['LeeSin', 'Rakan', 'Diana', 'Vayne', 'MasterYi', 'Nilah', 'Sion', 'Zed', 'Talon', 'Akshan', 'Zoe', 'Akali', 'Bard', 'Riven', 'Vi', 'Ashe', 'Syndra', 'Rell', 'Gragas', 'Darius', 'Malzahar', 'Soraka', 'Gwen', 'Graves', 'Qiyana', 'Azir', 'Viego', 'Ahri', 'Kayle', 'Veigar', 'Rammus', 'FiddleSticks', 'Morgana', 'TahmKench', 'Lillia', 'Jinx', 'Thresh', 'Warwick', 'Nasus', 'Fiora', 'Ziggs', 'Nautilus', 'Yasuo', 'Khazix', 'Sylas', 'Ivern', 'Kaisa', 'Shyvana', 'Nami', 'Orianna', 'Ezreal', 'Malphite', 'Mordekaiser', 'Brand', 'Leona', 'Shaco', 'Ryze', 'Nocturne', 'Rengar', 'Kled', 'Senna', 'Varus', 'Jhin', 'Singed', 'MonkeyKing', 'Aatrox', 'Vladimir', 'Belveth', 'Heimerdinger', 'Zilean', 'Skarner', 'Yuumi', 'Samira', 'Lissandra', 'Pantheon', 'Caitlyn', 'Braum', 'Kennen', 'Zac', 'Illaoi', 'Sona', 'Tristana', 'DrMundo', 'Seraphine', 'Viktor', 'Ornn', 'Cassiopeia', 'Garen', 'Yorick', 'Maokai', 'Sivir', 'Zeri', 'Tryndamere', 'Karma', 'Renata', 'Kassadin', 'Katarina', 'Vex', 'Corki', 'Amumu', 'Poppy'

In [5]:
img_rows=224
img_cols=224

In [6]:
num_channel=3

In [8]:
from sklearn import preprocessing

img_data_list=[]
labels_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(224,224))
        img_data_list.append(input_img_resize)
        labels_list.append(dataset)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
print (img_data.shape)

print (np.mean(img_data))
print (np.std(img_data))

print (img_data.mean(axis=0))
print (img_data.std(axis=0)) 

Loaded the images of dataset-LeeSin

Loaded the images of dataset-Rakan

Loaded the images of dataset-Diana

Loaded the images of dataset-Vayne

Loaded the images of dataset-MasterYi

Loaded the images of dataset-Nilah

Loaded the images of dataset-Sion

Loaded the images of dataset-Zed

Loaded the images of dataset-Talon

Loaded the images of dataset-Akshan

Loaded the images of dataset-Zoe

Loaded the images of dataset-Akali

Loaded the images of dataset-Bard

Loaded the images of dataset-Riven

Loaded the images of dataset-Vi

Loaded the images of dataset-Ashe

Loaded the images of dataset-Syndra

Loaded the images of dataset-Rell

Loaded the images of dataset-Gragas

Loaded the images of dataset-Darius

Loaded the images of dataset-Malzahar

Loaded the images of dataset-Soraka

Loaded the images of dataset-Gwen

Loaded the images of dataset-Graves

Loaded the images of dataset-Qiyana

Loaded the images of dataset-Azir

Loaded the images of dataset-Viego

Loaded the images of datase

In [9]:
img_data.shape


(1620, 224, 224, 3)

In [10]:
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
for i in range(0,num_classes):
    labels[i*10:(i+1)*10]=i

names = data_dir_list

In [11]:
Y = np_utils.to_categorical(labels, num_classes)
print(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(img_data, Y, test_size=0.2, random_state=2)
print(X_train.shape)


(1296, 224, 224, 3)


# Unitiliser ResNet en le réentrainant

In [13]:
# Import resnet50
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions

# Import load image from keras.utils
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

# Import Model from keras.models
from tensorflow.keras.models import Model


In [15]:
# Load the ResNet50 model
resnet_model = ResNet50(weights='imagenet')

# Change the last layer of the model
x = resnet_model.layers[-2].output
predictions = Dense(162, activation='softmax')(x)
model = Model(inputs=resnet_model.input, outputs=predictions)

# Freeze all the layers except the last 4 layers
for layer in model.layers[:-4]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))


Epoch 1/5
41/41 [==============================] - 275s 7s/step - loss: 3.2585 - accuracy: 0.5301 - val_loss: 0.2551 - val_accuracy: 0.9784
Epoch 2/5
41/41 [==============================] - 246s 6s/step - loss: 0.0483 - accuracy: 0.9985 - val_loss: 0.0134 - val_accuracy: 1.0000
Epoch 3/5
41/41 [==============================] - 251s 6s/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.0093 - val_accuracy: 1.0000
Epoch 4/5
41/41 [==============================] - 244s 6s/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0075 - val_accuracy: 1.0000
Epoch 5/5
41/41 [==============================] - 267s 7s/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000


In [16]:
# Save the model 
model.save('model_resnet.h5')   

In [28]:
# Predict champ2.jpg (resize it to 224x224)
img = load_img('./champ5.jpg', target_size=(224, 224))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

# Predict the class
pred = model.predict(img)
# take the 5 most probable classes
pred = np.argsort(pred[0])[-5:]
# name of the 5 most probable classes
for i in pred:
    print(names[i])





1/1 [==============================] - 0s 249ms/step
Viktor
Ahri
Zoe
Jayce
Fiora
